In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/checkpoint_11_df.csv")
df = df.astype(str)
import requests
import re
import json
import time
import datetime
from IPython.display import clear_output
!pip install demjson3
from demjson3 import decode

Mounted at /content/drive


In [ ]:
df["doi"].describe()

,doi
count,26806
unique,21579
top,nan
freq,5228


In [ ]:
# https://www.crossref.org/blog/using-the-crossref-rest-api.-part-12-with-europe-pmc/ - this though PMC? anyways, I dont have PMCs for all, and they are not available on OpenAlex for all of em, so might as well use DOI cause those are available on OpenAlex as long as the article is there (and its the largest database, so yeah ... only chance at that)
length = len(df.loc[(df["doi"] == "nan") & (df["references"] == "[]")])
iter = 0
def get_doi_of_nan(row):
  print(".", end="")
  if ( str(row["doi"]) != "nan" ):
    return row["doi"]

  if ( row["references"] != "[]" ):
    return row["doi"]

  global iter
  global length
  iter = iter + 1

  try:
    response = requests.get(f'https://api.openalex.org/works/pmid:{row["pmid"]}')
    decoded_json = decode(response.text)
    clear_output(wait=True)
    print(f'{iter}/{length} DONE PMID = {row["pmid"]},\tDOI = ', end="")
  except:
    print("nan - (DOI not found on OpenAlex)")
    return "nan"

  if ( decoded_json["doi"] == None ):
    print("nan - (DOI not found on OpenAlex)")
    return "nan"

  result = decoded_json["doi"].replace("https://doi.org/", "")
  print(result)
  return result

df["doi"] = df.apply( get_doi_of_nan, axis=1 )

4220/4220 DONE PMID = 21535973,	DOI = nan - (DOI not found on OpenAlex)


In [ ]:
df["doi"].describe()

,doi
count,21319
unique,21319
top,10.1371/journal.pone.0039966
freq,1


In [ ]:
# def change_row(row):
#   if ( str(row["doi"]) == "[]" ):
#     return "nan"

# df["doi"] = df.apply( change_row, axis=1 )

In [ ]:
df = df.astype(str)
df.to_csv("/content/drive/MyDrive/checkpoint_9_df.csv", encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/checkpoint_9_df.csv")
df = df.astype(str)
df["doi"].describe()

,doi
count,26806
unique,21579
top,nan
freq,5228


In [ ]:
import requests
import xml.etree.ElementTree as ET
import re
import datetime
from IPython.display import clear_output

# http://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0039966&type=manuscript
# https://api.crossref.org/works/10.1371/journal.pone.0032774.xml

def innertext(elt):
    return (elt.text or '') +' '.join(innertext(e)+(e.tail or '') for e in elt)

length=df.shape[0]
iter=0
avg_time_taken=0
remaining_hours=0
remaining_minutes=0
remaining_seconds=0
def citations(paper_doi):
  start = datetime.datetime.now()
  global iter
  global length
  global avg_time_taken
  global remaining_hours
  global remaining_minutes
  global remaining_seconds
  iter=iter+1
  # response = requests.get(f'http://journals.plos.org/plosone/article/file?id={paper_doi}&type=manuscript') # functions, but PLOS has almost none of the articles we have ...
  response = requests.get(f'https://api.crossref.org/works/{paper_doi}.xml')
  clear_output(wait=True)
  print( f'{iter}/{length}\t{paper_doi} - status: {response.status_code}' )
  if ( response.status_code != 200 ):
    return "[]"
  xml_tree = ET.ElementTree(ET.fromstring(response.text))
  refs = xml_tree.findall(".//{*}citation")
  ref_texts=[]
  for ref in refs:
    ref_text = ' '.join(innertext(ref).split())
    # print( ref.find(".//comment") ) # some have DOI of article
    if ( ref_text == "" ):
      continue
    ref_texts.append(ref_text)
  ref_texts = json.dumps(ref_texts)

  # print( ref_texts )
  end = datetime.datetime.now()
  time_length = end - start
  time_length = int(time_length.total_seconds() * 1000) / 1000
  avg_time_taken = (avg_time_taken * (iter-1) + time_length) / iter
  remaining = ( length - iter ) * avg_time_taken
  remaining_hours = int( remaining / 3600 )
  remaining_minutes = int( ( remaining % 3600 ) / 60 )
  remaining_seconds = int( remaining % 60 )
  print( f'Took {time_length} seconds ... {remaining_hours} hours, {remaining_minutes} minutes, {remaining_seconds} seconds remaining ... ' )
  if ( "[" not in ref_texts ):
    return "[]"
  return ref_texts


In [ ]:
import json
import time

for i in range(0, length):
  clear_output(wait=True)
  print(f'{i}/{length} ... ', end="")
  if ( str(df.iloc[i].at["references"]) != "[]" ):
    continue

  if ( str(df.iloc[i].at["doi"]) == "nan" ):
    continue

  try:
    print(f'DOI: {str(df.iloc[i].at["doi"])}')
    res = citations(df.iloc[i].at["doi"])
    print( f'\nOld df value:\t\t{df.loc[i, "references"]}' )
    print( f'\nReturned result:\t{res}' )
    df.loc[i, "references"] = res
    print( f'\nNew df value:\t\t{df.loc[i, "references"]}' )
    print( f'\nUnique references: {df["references"].nunique()}' )
    sleep(5)
  except:
    pass

# def get_citations(row):
#   res = json.dumps(citations(row["doi"]))
#   print( f'\nReturned result:\n{res}' )
#   return res

# df["references"] = df.apply( get_citations, axis=1 )


26805/26806 ... 

In [ ]:
# citations("10.1016/j.yexmp.2012.01.008")

In [ ]:
# import json

# with open("/content/drive/MyDrive/intermediary_citations.csv", "w") as fd:
#     fd.write("pmid,citations\n")
# # df_temp = pd.DataFrame( {"pmid":[], "citations":[]} )
# # df_temp.to_csv('/content/drive/MyDrive/intermediary_citations.csv', index=0)

# # def get_citations(row):
# length = df.shape[0]
# for i in range(0, length):
#   df_temp = pd.DataFrame( {"pmid":[], "citations":[]} )

#   res = json.dumps(citations(df["doi"].iloc[i]))
#   print( f'\nReturned result:\n{res}' )

#   # df_temp.loc[len(df_temp)] = [df["pmid"].iloc[i], res]
#   # df_temp.to_csv('/content/drive/MyDrive/intermediary_citations.csv', mode='a', header=False, index=0)
#   with open('/content/drive/MyDrive/intermediary_citations.csv', 'a') as fd:
#       fd.write(f'{df["pmid"].iloc[i]},{res}\n')

# # df["references"] = df.apply( get_citations, axis=1 )

In [ ]:
df = df.astype(str)
df.to_csv("/content/drive/MyDrive/checkpoint_10_df.csv", encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/checkpoint_10_df.csv")
df = df.astype(str)
df["doi"].describe()

,doi
count,26806
unique,21579
top,nan
freq,5228


In [ ]:
iter = 0
length = len(df)
def fix_broken_reference_json(row):
  global iter
  global length
  iter=iter+1
  clear_output(wait=True)
  print( f'{iter}/{length} ... ' )
  new = row["references"]
  try:
    ref_json = decode(row["references"])
  except:
    print("GETS")
    new = new.replace("\": \"", "\":\"")
    new = new.replace("\", \"", "\",\"")
    new = new.replace("\"}", "SPECIAL_REPLACER}").replace("{\"", "{SPECIAL_REPLACER").replace("\":\"", "SPECIAL_REPLACER2_DOTSSPECIAL_REPLACER").replace("\",\"", "SPECIAL_REPLACER,SPECIAL_REPLACER")
    # new = new.replace("\"", "\\\"")
    new = new.replace("\"", "")
    new = new.replace(":", "")
    new = "".join(new.splitlines())
    new = new.replace("SPECIAL_REPLACER}", "\"}").replace("{SPECIAL_REPLACER", "{\"").replace("SPECIAL_REPLACER2_DOTSSPECIAL_REPLACER", "\":\"").replace("SPECIAL_REPLACER,SPECIAL_REPLACER", "\",\"")
    time.sleep(1)
    try:
      json=decode(new)
    except:
      # clear_output(wait=True)
      print( f"OLD: {row['references']}" )
      print( f"NEW: {new}" )
      print ("Theres an error, check an online json validator ... ")
      time.sleep(1000000000000000000000);
      exit()
  return new

df["references"] = df.apply(fix_broken_reference_json, axis=1)

26806/26806 ... 


In [ ]:
df = df.astype(str)
df.to_csv("/content/drive/MyDrive/checkpoint_11_df.csv", encoding='utf-8', index=False)

In [ ]:
!pip install demjson3
from demjson3 import decode

# maybe tryna get the DOIs frm online cause most aint there ... use pmid to get doi for those too maybe ...
def reformat_pubmed_references(row):
  # try:
  print(row["references"])
  ref_json = decode(row["references"])
  if (len(ref_json)==0):
    return row["references"]

  if 'article_ids' in row["references"]:
    clear_output(wait=True)
    print("OLD: ", end="")
    print(row["references"])
    new_refs = []
    for i in range(0, len(ref_json)):
      new_ref = {}
      # print(ref_json[i])
      if '"doi"' in json.dumps(ref_json[i]["article_ids"]):
        # print("THERES ONE")
        new_ref[ "doi" ] = ref_json[i]["article_ids"]["doi"]
        # print(new_ref)
        # exit()
        # time.sleep(1000000000000000000)
      else:
        new_ref[ "doi" ] = "nan"

      new_ref[ "citation" ] = ref_json[i]["citation"]
      new_refs.append(new_ref)

    print("NEW: ", end="")
    print(new_refs)
    return json.dumps(new_refs)
  else:
    return row["references"]
  # except:
  #   return row["references"]

df["references"] = df.apply(reformat_pubmed_references, axis=1)

OLD: [{"citation": "Vardulaki KA, Prevost TC, Walker NM, et al. Incidence among men of asymptomatic abdominal aortic aneurysms: estimates from 500 screen detected cases. J Med Screen 1999;6:50\u20134.", "article_ids": {"pubmed": "10321373"}}, {"citation": "Assar AN, Zarins CK. Ruptured abdominal aortic aneurysm: a surgical emergency with many clinical presentations. Postgrad Med J 2009;85:268\u201373.", "article_ids": {"pubmed": "19520879"}}, {"citation": "Ghilardi G, Scorza R, Bortolani E, et al. Rupture of abdominal aortic aneurysms into the major abdominal veins. J Cardiovasc Surg (Torino) 1993;34:39\u201347.", "article_ids": {"pubmed": "8482703"}}, {"citation": "Cinara IS, Davidovic LB, Kostic DM, et al. Aorto-caval fistulas: a review of eighteen years experience. Acta Chir Belg 2005;105:616\u201320.", "article_ids": {"pubmed": "16438071"}}, {"citation": "Combe J, Besancenot J, Milleret P, et al. Iliocaval venous compression due to aneurysm of the abdominal aorta: report of ten cas

In [ ]:
df = df.astype(str)
df.to_csv("/content/drive/MyDrive/checkpoint_12_df.csv", encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/checkpoint_11_df.csv")

In [ ]:
def see_old_pubmed(row):
  clear_output(wait=True)
  if ( "article_id" in row["references"] ):
    print(row["references"])
  # else:
  #   print("no article_id")

df.apply(see_old_pubmed, axis=1)

[{"citation": "Bertoglio K, Hendren RL. New developments in autism. The Psychiatric clinics of North America. 2009;32(1):1\u201314.", "article_ids": {"pubmed": "19248913"}}, {"citation": "Ashwood P, Wills S, Van de Water J. The immune response in autism: a new frontier for autism research. Journal of leukocyte biology. 2006;80(1):1\u201315.", "article_ids": {"pubmed": "16698940"}}, {"citation": "Vargas DL, Nascimbene C, Krishnan C, Zimmerman AW, Pardo CA. Neuroglial activation and neuroinflammation in the brain of patients with autism. Annals of neurology. 2005;57(1):67\u201381.", "article_ids": {"pubmed": "15546155"}}, {"citation": "Cohly HH, Panja A. Immunological findings in autism. International review of neurobiology. 2005;71:317\u201341.", "article_ids": {"pubmed": "16512356"}}, {"citation": "Gomes FC, Sousa Vde O, Romao L. Emerging roles for TGF-beta1 in nervous system development. International journal of developmental neuroscience : the official journal of the International So

In [ ]:
df["references"].describe()

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/checkpoint_4_df.csv")

Mounted at /content/drive


In [ ]:
import re
import json
from IPython.display import clear_output

def get_doi( row ):
  clear_output(wait=True)
  try:
    print( row["references"] )
    citations = json.loads(row["references"])
    # citations = ['Cheney Anesthesiology 2006 10.1097/00000542-200612000-00007 105 1081', 'Peterson Anesthesiology 2005 10.1097/00000542-200507000-00009 103 33', 'Mallampati Can Anaesth Soc J 1985 10.1007/BF03011357 32 429', 'Samsoon Anaesthesia 1987 10.1111/j.1365-2044.1987.tb04039.x 42 487', 'Wilson Br J Anaesth 1988 10.1093/bja/61.2.211 61 211', 'Shiga Anesthesiology 2005 10.1097/00000542-200508000-00027 103 429', 'Cormack Anaesthesia 1984 10.1111/j.1365-2044.1984.tb08932.x 39 1105', 'Adnet Anesthesiology 1997 10.1097/00000542-199712000-00005 87 1290']
    citations_res = []
    for citation in citations:
      doi = re.search("^.*(10\.[A-Za-z0-9.\/-]+)(?<!\.)(?=[ ]|\.).*$", citation)
      if ( doi == None ):
        continue
      doi = doi.group(1)
      citations_doi = {}
      if ( doi != "" ):
        citations_doi["doi"]=doi
      else:
        citations_doi["doi"]="no_doi"

      citations_doi["citation"]=citation
      citations_res.append( citations_doi )

    # print(json.dumps(citations_res, indent=4))
    citations_res = json.dumps(citations_res) # convert to string ...
    print( citations_res )
    return citations_res
  except:
    return "[]"

df["references"] = df.apply( get_doi, axis=1 )

nan


In [ ]:
df = df.astype(str)
df.to_csv("/content/drive/MyDrive/checkpoint_5_df.csv", encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/checkpoint_5_df.csv")

In [ ]:
df["references"].describe()

,references
count,26806
unique,16745
top,[]
freq,10055


In [ ]:
df_1 = pd.read_csv("/content/drive/MyDrive/checkpoint_3_df.csv")

In [ ]:
df_1["references"].describe()

,references
count,26806
unique,6252
top,[]
freq,20554


In [ ]:
def change_empty(row):
  if ( row["references"] == "[]" ):
    pmid=row["pmid"]
    row_iter = df_1.index[df_1['pmid'] == pmid].tolist()[0]
    # print( df_1["references"].iloc[[row_iter]].tolist()[0] )
    return df_1["references"].iloc[[row_iter]].tolist()[0]
  return row["references"]

df["references"] = df.apply( change_empty, axis=1 )

In [ ]:
df["references"].describe()

,references
count,26806
unique,17980
top,[]
freq,8820


In [ ]:
df["references"].describe()
df.to_csv("/content/drive/MyDrive/checkpoint_6_df.csv", encoding='utf-8', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/checkpoint_7_df.csv")

Mounted at /content/drive


In [ ]:
import requests
import re
import json
import time
import datetime
from IPython.display import clear_output
!pip install demjson3
from demjson3 import decode

# https://github.com/eschares/OpenAlex-CitedReferences/blob/main/notebooks/1-Pull_the_data_OpenAlex-citedreferences.ipynb
# https://groups.google.com/g/openalex-users/c/O4kidbDVFtI?pli=1

# http://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0039966&type=manuscript
# https://api.crossref.org/works/10.1371/journal.pone.0032774.xml

def innertext(elt):
    return (elt.text or '') +' '.join(innertext(e)+(e.tail or '') for e in elt)

def alex_citation(paper_pmid):
  # https://api.openalex.org/works/pmid:22815718&doi:10.1371/journal.pone.0039966
  response = requests.get(f'https://api.openalex.org/works/pmid:{paper_pmid}')
  # response = requests.get(f'https://api.openalex.org/works/mailto=apaluza3@gmail.com/https://doi.org/{paper_doi}')
  # response = requests.get(f'https://api.openalex.org/works/mailto=apaluza3@gmail.com/doi:{paper_doi}')
  print( f'Computing PMID: {paper_pmid} ... (status: {response.status_code})' )
  decoded_json = decode(response.text)
  # print(type(decoded_json))
  # print(decoded_json)
  # citations = response.json()["results"][0]["referenced_works"]
  citations = decoded_json["referenced_works"]

  citation_list = ""
  if ( len(citations) == 0 ):
    return "[]"

  citation_iter = 0
  for citation in citations:
    citation_iter = citation_iter + 1
    if ( citation_iter == 100 ):
      break # maximum request limit ...
    citation = citation.replace("https://openalex.org/", "")
    citation_list = citation_list + citation + '|'
  citation_list = citation_list[: -1]

  url = "https://api.openalex.org/works?filter=openalex:"+citation_list
  print(url)
  cit_json = decode(requests.get(url).text)

  full_jsons = []
  # print( len(cit_json["results"]) )
  for citation in cit_json["results"]:
    title = citation["title"]
    authors=""
    for author in citation["authorships"]:
      # print(author["raw_author_name"])
      authors = authors + author["raw_author_name"] + ", "
    authors = authors[: -2]
    year = citation["publication_year"]
    doi = citation["doi"]
    full_json = { "doi": doi, "citation": f'{authors} ({year}) {title} {doi}' }
    full_jsons.append(full_json)
  return json.dumps(full_jsons)

# result_to_convert = alex_citation("10.1038/jhh.2011.8")
# final_result = str.encode(result_to_convert).decode("unicode_escape")

length=len(df[ df["references"] == "[]" ])
iter=0
avg_time_taken=0
remaining_hours=0
remaining_minutes=0
remaining_seconds=0
def alex_citations(row):
  if ( row["references"] != "[]" ):
    return row["references"]

  # if ( str(row["doi"]) == "nan" ):
  #   return row["references"]

  start = datetime.datetime.now()
  global iter
  global length
  global avg_time_taken
  global remaining_hours
  global remaining_minutes
  global remaining_seconds
  iter=iter+1
  try:
    result_to_convert = alex_citation(row["pmid"])
    final_result = str.encode(result_to_convert).decode("unicode_escape")
  except:
    clear_output(wait=True)
    print( "Some exception happened with the previous DOI ... " )
    return "[]"
  clear_output(wait=True)

  end = datetime.datetime.now()
  time_length = end - start
  time_length = int(time_length.total_seconds() * 1000) / 1000
  avg_time_taken = (avg_time_taken * (iter-1) + time_length) / iter
  remaining = ( length - iter ) * avg_time_taken
  remaining_hours = int( remaining / 3600 )
  remaining_minutes = int( ( remaining % 3600 ) / 60 )
  remaining_seconds = int( remaining % 60 )
  print( f'{iter}/{length} - DONE' )
  print( f'Took {time_length} seconds ... {remaining_hours} hours, {remaining_minutes} minutes, {remaining_seconds} seconds remaining ... ' )
  print( f'OLD: {row["references"]}' )
  print( f'NEW: {final_result}' )
  print( f'\n{iter+1}/{length} - now computing ...' )
  time.sleep(0.1)
  return final_result

df["references"] = df.apply( alex_citations, axis=1 )

6921/6921 - DONE
Took 0.246 seconds ... 0 hours, 0 minutes, 0 seconds remaining ... 
OLD: []
NEW: []

6922/6921 - now computing ...


In [ ]:
df.to_csv("/content/drive/MyDrive/checkpoint_8_df.csv", encoding='utf-8', index=False)

In [ ]:
df["references"].describe()

,references
count,26806
unique,20942
top,[]
freq,5856


In [ ]:
df["pmid"]

,pmid
0,22815718
1,27286562
2,22381459
3,21396219
4,24167451
...,...
26801,16302942
26802,9855106
26803,22004635
26804,17126784


In [ ]:
# Here is code for SCOPUS ... it does NOT let me get citations by default, but I could email the API Support Team to let me get citations for free, but there's still a weekly quota ... so best not to use it ...

# !pip install pyscopus==1.0.3a2
# import pyscopus
# scopus = pyscopus.Scopus(key)
# response_json = json.loads(scopus.search(f'doi({doi})', view='STANDARD').to_json(orient="records"))
# response_json
# import json
# import httpx
# import time
# !pip install demjson3
# from demjson3 import decode

# key = 'aa8e9c8989580dd7fb3f805ea5d42d65' # for scopus api
# doi = '10.1371/journal.pone.0039966'

In [ ]:
# def get_eid(paper_doi, apikey):
#     apikey=apikey
#     headers={
#         "X-ELS-APIKey":apikey,
#         "Accept":'application/json'
#          }
#     timeout = httpx.Timeout(10.0, connect=60.0)
#     client = httpx.Client(timeout=timeout,headers=headers)
#     query="&view=FULL"
#     url=f'https://api.elsevier.com/content/search/scopus?query=doi({paper_doi})'
#     r=client.get(url)
#     return r

# def get_citations(paper_eid, apikey):
#     apikey=apikey
#     headers={
#         "X-ELS-APIKey":apikey,
#         "Accept":'application/json'
#          }
#     timeout = httpx.Timeout(10.0, connect=60.0)
#     client = httpx.Client(timeout=timeout,headers=headers)
#     query="&view=FULL"
#     # url='https://api.elsevier.com/content/search/scopus?query=refeid('+paper_eid+')'
#     # https://api.elsevier.com/content/abstract/citations?doi=10.1016%2FS0014-5793(01)03313-0&apiKey=7f59af901d2d86f78a1fd60c1bf9426a&httpAccept=application%2Fjson
#     # url = "https://api.elsevier.com/content/abstract/citations?doi="+paper_doi+"&apiKey="+apikey+"&httpAccept=application%2Fjson"


#     url = f'https://api.elsevier.com/content/abstract/EID:{paper_eid}?apiKey={apikey}&view=REF'
#     r=client.get(url)
#     print(url)
#     return r


# def get_full_text(paper_doi, apikey):
#     apikey=apikey
#     headers={
#         "X-ELS-APIKey":apikey,
#         "Accept":'application/json'
#          }
#     timeout = httpx.Timeout(10.0, connect=60.0)
#     client = httpx.Client(timeout=timeout,headers=headers)
#     query="&view=FULL"
#     url="https://api.elsevier.com/content/article/doi/"+paper_doi
#     r=client.get(url)
#     return r


# # Get document
# eid_response = get_eid(doi, key)
# eid = decode(eid_response.text)["search-results"]["entry"][0]["eid"]

# print(eid)
# citations = get_citations( eid, key )
# # Parse document
# json_acceptable_string = citations.text
# d = decode(json_acceptable_string)
# # Print document
# print(d)

In [ ]:
# !pip install pybliometrics
# import pybliometrics
# # Document-specific information
# from pybliometrics.scopus import AbstractRetrieval, CitationOverview
# pybliometrics.scopus.init(config_dir="/content/drive/MyDrive/config.txt")
# ab = CitationOverview([doi])

In [ ]:
# !pip install pyscopus
# import pyscopus
# scopus = pyscopus.Scopus(key)
# response_json = scopus.search(f'doi({doi})', view='STANDARD')
# response_json

,scopus_id,title,publication_name,issn,isbn,eissn,volume,page_range,cover_date,doi,citation_count,affiliation,aggregation_type,subtype_description,authors,full_text
0,84863614148,The diagnostic accuracy of urine-based Xpert M...,PLoS ONE,None,None,19326203,7,None,2012-07-09,10.1371/journal.pone.0039966,74,[{'name': 'University of Cape Town Lung Instit...,Journal,Article,[],None
